# The Deep Learning Book (Simplified)
## Part II - Modern Practical Deep Networks
*This is a series of blog posts on the [Deep Learning book](http://deeplearningbook.org)
where we are attempting to provide a summary of each chapter highlighting the concepts 
that we found to be most important so that other people can use it as a starting point
for reading the chapters, while including the code for reproducing some of the results. 
Please refer [this](http://www.deeplearningbook.org/contents/notation.html) for more clarity on 
notation.*


## Chapter 7: Regularization for Deep Learning

Recalling from Chapter 5, **overfitting** is said to occur when the training error keeps decreasing but the test error (or the generalization error) starts increasing. **Regularization** is the modification we make to a learning algorithm that reduces its generalization error, but not its training error. There are various ways of doing this, some of which include restriction on parameter values or adding terms to the objective function, etc.

These constraints are designed to encode some sort of prior knowledge, with a preference towards simpler models to promote generalization (See [Occam's Razor](https://en.wikipedia.org/wiki/Occam%27s_razor)). The sections present in this chapter are listed below: <br>

**1. Parameter Norm Penalties** <br>
**2. Norm Penalties as Constrained Optimization** <br>
**3. Regularization and Under-Constrained Problems** <br>
**4. Dataset Augmentation** <br>
**5. Noise Robustness** <br>
**6. Semi-Supervised Learning** <br>
**7. Mutlitask Learning** <br>
**8. Early Stopping** <br>
**9. Parameter Tying and Parameter Sharing** <br>
**10. Sparse Representations** <br>
**11. Bagging and Other Ensemble Methods** <br>
**12. Dropout** <br>
**13. Adversarial Training** <br>
**14. Tangent Distance, Tangent Prop and Manifold Tangent Classifier** <br>

### 1. Parameter Norm Penalties

The idea here is to limit the capacity (the space of all possible model families) of the model 
by adding a parameter norm <br>
penalty, $\Omega(\theta)$, to the objective function, $J$:

$$ \tilde{J}(\theta; X, y) =  J(\theta; X, y) + \lambda \Omega(\theta)$$

Here, $\theta$ represents only the weights and not the biases, the reason being that the biases require much less data to fit and do not add much variance.

**1.1 $L^2$ Parameter Regularization**

Here, the parameter norm penalty:
$$\Omega(\theta) = \frac {||w||_2^2} {2}$$

This makes the objective function:

$$ \tilde{J} (\theta; X, y) = J(\theta; X, y) + \alpha \frac {w^T w} {2} $$

Applying the 2nd order Taylor-Series approximation at the point $w^*$ where $\tilde{J} (\theta; X, y)$ assumes the minimum value, i.e., $\bigtriangledown_w \tilde {J} (w^*) = 0$:

$$ \hat{J}(w) = J(w^*) + \frac{(w - w^*)^T H(J(w^*))(w - w^*)} {2} $$

Finally, $\bigtriangledown_w \hat{J}(w) = H(J(w^*))(w - w^*)$ and the overall gradient of the objective function becomes:

$$ \bigtriangledown_w \tilde{J}(w) = H(J(w^*))(\tilde{w} - w^*) + \alpha \tilde{w} = 0$$
$$ \tilde{w} = (H + \alpha I)^{-1} H w^* $$

As $\alpha$ approaches 0, $w$ comes closer to $w^*$. Finally, since $H$ is real and symmetric, it can be decomposed into a diagonal matrix $\wedge$ and an orthonormal set of eigenvectors, $Q$. That is, $H = Q^T\wedge Q$.

![l2 reg](images/L2_reg.png)

Because of the marked term, the value of each weight is rescaled along the eigenvectors of $H$. The value of the weights along the $i^{th}$ eigenvector is rescaled by $\frac {\lambda_i}{\lambda_i + \alpha}$, where $\lambda_i$ represents the eigenvalue corresponding to the $i^{th}$ eigenvector.

| Condition| Effect |
| --- | --- |
|  $\lambda_i >> \alpha$ | not much effect |
|  $\lambda_i << \alpha$ | The weight value almost shrunk to zero |

The diagram below illustrates this well.

![L2 scaling](images/L2_scaling.png)

To look at its application to Machine Learning, we have to look at linear regression. The objective function there is exactly quadratic, given by:

![linear_reg](images/linear_reg.png)

**1.2 $L^1$ parameter regularization**

Here, the parameter norm penalty:
$$\Omega(\theta) = ||w||_1 $$

Making the gradient of the overall objective function:

$$ \bigtriangledown_w \tilde{J}(\theta; X, y) = \bigtriangledown_w J(\theta; X, y) + \alpha * sign(w) $$

Now, the last term, sign(w), create a difficulty that the gradient no longer scales linearly with $w$. This leads to a few complexities in arriving at the optimal solution (which I am going to skip):
![l1_reg](images/l1_reg.png)

Our current interpretation of the `max` term is that, there shouldn't be a zero crossing, as the gradient of the absolute value function is not differentiable at zero.

![lasso result](images/lasso_result.png)


Thus, $L^1$ regularization has the property of sparsity, which is its fundamental distinguishing feature from $L^2$. Hence, $L^1$ is used for feature selection as *LASSO*.

### 2. Norm penalties as constrained optimization

From chapter 4's section 4, we know that to minimize any function under some constraints, we can construct a generalized Lagrangian function containing the objective function along with the penalties. Suppose we wanted $\Omega(\theta)) < k$, then we could construct the following Lagrangian:
![lagrangian](images/lagrangian.png)

Thus, $\theta^* = argmin_{\theta} (max_{\alpha, \alpha >= 0} \hspace{.2cm} \mathcal{L}(\theta, \alpha; X, y))$. If  $\Omega(\theta) > k$, then $\alpha$ should be large to reduce its value below k. <br>
Likewise, if $\Omega(\theta) < k$, then $\alpha$ should be small. Assuming $\alpha$ to be a constant $\alpha^{*}$:

$$ \theta^* = argmin_{\theta} \hspace{.2cm} J(\theta; X, y) + \alpha^* \Omega(\theta)$$

This is now similar to the parameter norm penalty regularized objective function. Thus, parameter norm penalties naturally impose a constraint, like the L2-regularization defining a constrained L2-ball. Larger $\alpha$ means a smaller constrained region and vice versa. The idea of constraints over penalties, is important for several reasons. Penalties might cause non-convex optimization algorithms to get stuck in local minima due to small values of $\theta$, leading to the formation of so-called `dead cells`, as the weights entering and leaving them are too small to have an impact. Constraints don't enforce the weights to be near zero, rather being confined to a constrained region.

Another reason is that constraints induce higher stability. With higher learning rates, there might be a large weight, leading to a large gradient, which could go on iteratively leading to numerical overflow in the value of $\theta$. Constrains along with reprojection (to the corresponding ball) prevent the weights from becoming too large, thus, maintaining stability. 

A final suggestion made by Hinton was to restrict the individual column norms of the weight matrix rather than the Frobenius norm of the entire weight matrix, so as to prevent any hidden unit from having a large weight.